In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import subprocess

# Start the Ollama server as a background process
process = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("Ollama server started in the background.")
# You might want to add a short sleep here to ensure the server starts up
import time
time.sleep(5)

In [ ]:
!pip install nest_asyncio

In [ ]:
!pip install ollama -q

In [ ]:
# CELL 1: Start Ollama Server
import os

print("Starting Ollama server...")
# Run ollama serve in the background and redirect output to a log file
os.system("ollama serve > ollama_server.log 2>&1 &")
print("Ollama server started in the background. Check ollama_server.log for status.")

# Give the server a moment to start up
import time
time.sleep(10) # Wait 10 seconds (adjust if needed)

# Optional: Verify the server is running by listing models
print("Checking available models...")
!ollama list

In [ ]:
import ollama
import csv
import os
import time
import asyncio
import base64
from pathlib import Path
import nest_asyncio # Import nest_asyncio

# --- Configuration ---
# Replace with the identifier of your loaded model in Ollama
MODEL_IDENTIFIER = "/kaggle/input/customMode/qwen2-vl:7b" # Example model identifier, replace with your actual model
# Path to your input CSV file (must contain 'id' and 'file_name' columns)
INPUT_CSV_PATH = "/kaggle/input/tammathon-task-3/test.csv"
# Path where the output CSV with captions will be saved
OUTPUT_CSV_PATH = "/kaggle/working/output_with_captions.csv"
# Base folder where the images are located
IMAGE_BASE_FOLDER = Path("/kaggle/input/tammathon-task-3/test/test") # Use pathlib for better path handling
# The text prompt to send along with each image
PROMPT = "Only answer with \"Car image with [elements]\" for all image upload using elements within ['broken lamp', 'crack', 'dent', 'flat tire', 'scratch', 'shattered glass'] ONLY. make sure the car image contains those features. Dont put dent on everything until you are sure.\nie.\"Car image with flat tire, scratch.\""
# Optional delay between API calls in seconds (to avoid overwhelming the server)
DELAY_BETWEEN_REQUESTS_S = 1.0 # Delay in seconds

# --- Helper Function to Read CSV ---
def read_csv(file_path: str) -> tuple[list[dict], list[str]]:
    data = []
    fieldnames = []
    try:
        with open(file_path, mode='r', encoding='utf-8', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            fieldnames = reader.fieldnames
            if not fieldnames:
                 raise ValueError("CSV file appears to be empty or missing headers.")
            if 'id' not in fieldnames or 'file_name' not in fieldnames:
                raise ValueError("Input CSV must contain 'id' and 'file_name' columns.")
            for row in reader:
                data.append(row)
        if not data:
             print(f"Warning: CSV file '{file_path}' is empty or has no data rows after the header.")
    except FileNotFoundError:
        print(f"Error: Input CSV file not found at {file_path}")
        raise
    except Exception as e:
        print(f"Error reading CSV file {file_path}: {e}")
        raise
    return data, fieldnames

# --- Helper Function to Write CSV ---
def write_csv(file_path: str, data: list[dict], fieldnames: list[str]):
    try:
        with open(file_path, mode='w', encoding='utf-8', newline='') as csvfile:
            output_fieldnames = list(fieldnames)
            if 'caption' not in output_fieldnames:
                output_fieldnames.append('caption')
            writer = csv.DictWriter(csvfile, fieldnames=output_fieldnames, extrasaction='ignore')
            writer.writeheader()
            writer.writerows(data)
    except Exception as e:
        print(f"Error writing CSV file {file_path}: {e}")
        raise

# --- Main Asynchronous Script Logic ---
async def main():
    print("--- Starting Image Captioning Script (Python) ---")
    print(f"Model Identifier: {MODEL_IDENTIFIER}")
    print(f"Input CSV: {INPUT_CSV_PATH}")
    print(f"Image Folder: {IMAGE_BASE_FOLDER}")
    print(f"Output CSV: {OUTPUT_CSV_PATH}")

    if not os.path.exists(INPUT_CSV_PATH):
        print(f"Error: Input CSV not found at {INPUT_CSV_PATH}. Exiting.")
        return

    if not IMAGE_BASE_FOLDER.is_dir():
        print(f"Error: Image base folder not found or is not a directory at {IMAGE_BASE_FOLDER}. Exiting.")
        return

    data: list[dict] = []
    original_fieldnames = []

    try:
        print(f"Loading CSV file: {INPUT_CSV_PATH}...")
        data, original_fieldnames = read_csv(INPUT_CSV_PATH)
        if not data:
             print("CSV loading resulted in no data. Exiting.")
             return
        if not original_fieldnames:
             print("Could not read headers from CSV. Exiting.")
             return

        print(f"Found {len(data)} data rows in the CSV.")

        print("Initializing Ollama client...")
        client = ollama.AsyncClient()
        print("Ollama client initialized.")

        total_rows = len(data)
        print("Processing images...")

        for i, row in enumerate(data):
            image_filename = str(row.get('file_name', '')).strip()
            image_id = row.get('id', 'N/A')

            if not image_filename:
                print(f"Skipping row {i + 1} (ID: {image_id}) due to missing 'file_name'.")
                row['caption'] = None
                continue

            full_image_path = IMAGE_BASE_FOLDER / image_filename
            print(f"Processing row {i + 1}/{total_rows}: id={image_id}, file={image_filename}")
            caption: str | None = None

            if not full_image_path.is_file():
                print(f"  Warning: Image not found at {full_image_path}. Skipping.")
            else:
                try:
                    print(f"  Sending request to Ollama for {image_filename}...")
                    response = await client.chat(
                        model=MODEL_IDENTIFIER,
                        messages=[
                            {
                                'role': 'user',
                                'content': PROMPT,
                                'images': [str(full_image_path)]
                            }
                        ]
                    )
                    print("  Received response from Ollama.")

                    if response and 'message' in response and 'content' in response['message']:
                        caption = response['message']['content'].strip()
                        print(f"  Generated Caption (ID: {image_id}): {caption}")
                    else:
                        print(f"  Warning: Could not extract content from response for {image_filename}. Response: {response}")

                except Exception as e:
                    print(f"  Error processing {image_filename} with Ollama: {e}")

            row['caption'] = caption

            if DELAY_BETWEEN_REQUESTS_S > 0 and i < total_rows - 1:
                await asyncio.sleep(DELAY_BETWEEN_REQUESTS_S)

        print("\nFinished processing all images.")
        print(f"Saving results to: {OUTPUT_CSV_PATH}...")
        write_csv(OUTPUT_CSV_PATH, data, original_fieldnames)
        print("Output CSV saved successfully.")

    except Exception as e:
        print(f"\n--- An unexpected error occurred in the main script ---")
        print(f"Error: {e}")

    print("--- Script Finished ---")

# --- Run the main asynchronous function ---
if __name__ == "__main__":
    nest_asyncio.apply()
    # Now run the main asynchronous function
    asyncio.run(main())